# Agents

The RAG workflow essentially feeds an LLM with relevant data, but in a given scenario a user query might go beyond the scope of the tool that we have developed to collect the data. In such a scenario a one dimensional RAG workflow might be...

1) Input query.

2) Get closest matches from vector store.

3a) If information from vector store answers the question, then respond.

3b) If information from the vector store does not answer the question, then explain that no response is possible.

With Agentic RAG we look to equip the LLM with a series of tools. A single tool might be one document, say the procedure manual for a particular work flow. Let's say there are 8 of these. Each one of these tools has a description eg. "Procedurues for Process A." Now when this is queried the query is recieved, the LLM checks which tool might be useful, on the basis of this analysis the LLM conducts the search in the appropriate vector store. The LLM may choose to use any number of these tools, each time checking to see if information allows the question to be answered. As an ultimate fallback the LLM might get the best results from the internet and make an attempt to answer the question.

As an extension to this chain of reasoning logic can allow the LLM to break up the question, so that the total question is a sequential series of sub-questions. These subquestions might be answered with the help of numerous tools, the LLM step through each subquestion getting information until a final answer to the query can be made.

Here as a first step in this process we take two such tools, two documents about Excel, one is the basic operations in Excel, and the other explain the improvements made in the 2010 version. Now given a query the LLM will decide which tool (if any) it should use.





## $\color{blue}{Sections:}$
* Admin
* Chain
* Agent


---
## $\color{blue}{Admin}$
---

In [ ]:
%%capture
pip install langchain

In [ ]:
%%capture
pip install langchainhub

In [ ]:
%%capture
pip install langchain-openai

In [ ]:
%%capture
pip install langchain-community

In [ ]:
%%capture
!pip install faiss-cpu

In [ ]:
from langchain.chains import RetrievalQA, LLMChain
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.callbacks import StdOutCallbackHandler
from langchain_community.vectorstores import FAISS
from langchain.agents import Tool, ZeroShotAgent, AgentExecutor,create_react_agent
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain import hub
import getpass

In [ ]:
from google.colab import drive

In [ ]:
drive.mount("/content/drive")
%cd '/content/drive/MyDrive/'

Mounted at /content/drive
/content/drive/MyDrive


---
## $\color{blue}{Chain}$
---

Essentially we need three things in the chain.

* Vector store where we will get our context info.
* Prompt Template that defines how our prompt context info is formatted.
* LLM to answer the question.

In [ ]:
#''

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass('Your OpenAI API key please: ')

In [ ]:
llm = ChatOpenAI(model='gpt-3.5-turbo')
embeddings = OpenAIEmbeddings()
handler = StdOutCallbackHandler()

In [ ]:
# pdfs transformed to databases in notebook 3, reload them

# db_excel_2010 = FAISS.load_local("RAG_tutorial/dbs/excel_train.db",embeddings,allow_dangerous_deserialization=True)
# db_excel_basics = FAISS.load_local("RAG_tutorial/dbs/excel_basics.db",embeddings,allow_dangerous_deserialization=True)

In [ ]:
db_excel_2010

In [ ]:
retriever_2010 = db_excel_2010.as_retriever()
retriever_basics = db_excel_basics.as_retriever()

In [ ]:
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [ ]:
retrieval_qa_chat_prompt


ChatPromptTemplate(input_variables=['context', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'langchain-ai', 'lc_hub_repo': 'retrieval-qa-chat', 'lc_hub_commit_hash': 'b60afb6297176b022244feb83066e10ecadcda7b90423654c4a9d45e7a73cebc'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='Answer any use questions based solely on the context below:\n\n<context>\n{context}\n</context>')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

In [ ]:
combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)

In [ ]:
retrieval_chain_2010 = create_retrieval_chain(retriever_2010, combine_docs_chain)
retrieval_chain_basics = create_retrieval_chain(retriever_basics, combine_docs_chain)

---
## $\color{blue}{Agent}$
---

Each single tool that the LLM can use is going to be defined as an instance of the Langchain agents Tool class. Each Tool will require a name,a nd a description and a function. The function will take a query and return a response. As the tools we have created require the invoke method, we define functions that wrap this formatting step.

In [ ]:
def query_2010(input):
    return retrieval_chain_2010.invoke({'input': input})

In [ ]:
def query_basics(input):
    return retrieval_chain_basics.invoke({'input': input})

In [ ]:
tools = [
    Tool(
        name = "Excel2010",
        func= query_2010,
        description="Useful for understanding the new features released in MS Excel 2010."
    ),
    Tool(
        name = "ExcelBasics",
        func=query_basics,
        description="Useful for explaining how to perform basic operations in MS Excel."
    ),
]

This prompt template will have access to the input, the tools, and the agent scratchpad represents info learnt along the way by the LLM.

In [ ]:
prompt = hub.pull("hwchase17/react")

In [ ]:
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [ ]:
prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

In [ ]:
excel_agent = create_react_agent(llm, tools, prompt)

In [ ]:
#excel_agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
excel_agent_chain = AgentExecutor.from_agent_and_tools(agent=excel_agent, tools=tools, verbose=True,return_intermediate_steps=True)

In [ ]:
r = excel_agent_chain.invoke({'input': 'I need to format numbers in basic Excel,  and use new version of Excel to discover patterns and trends in my data'})



> Entering new AgentExecutor chain...
I should first learn how to format numbers in basic Excel and then explore the new features in Excel 2010 to analyze patterns and trends in my data.
Action: ExcelBasics
Action Input: Learn how to format numbers in Excel{'input': 'Learn how to format numbers in Excel', 'context': [Document(page_content='1. Select the range to format  \n2. Click on the HOME  tab, then click on the \ndrop arrow for Number  Format  in the \nNumber  group  \n3. Click on the desired number format  \n7 \nHandy to Know…  \n\uf0b7Excel may appear to round values up or \ndown as necessary – however, the value in \nthe cell does not change. Sometimes you’ll \nsee minor rounding discrepancies.  \n\uf0b7The Currency  format shows the currency \nformat and symbol appropriate to the country \nyour computer is configured for.', metadata={'source': './pdfs/excel_valid.pdf', 'page': 20}, _lc_kwargs={'page_content': '1. Select the range to format  \n2. Click on the HOME  tab, then 

In [ ]:
# input/ output/ intermediate_steps
question = f"Initial query... {r['input']}\n\n"
response = f"Final response... {r['output']}\n\n"
hold = ""
for i in range(len(r['intermediate_steps'])):
  reflection = r['intermediate_steps'][i][0].log.split('\n')[0]
  tool = r['intermediate_steps'][i][0].tool
  tool_in = r['intermediate_steps'][i][0].tool_input
  resp = r['intermediate_steps'][i][1]['answer']
  hold += f'Step {i+1}:\n'
  hold += f'The agent considers... "{reflection}"\n'
  hold += f'The angent chooses the tool...  "{tool}."\n'
  hold += f'The angent queries the tool... "{tool_in}."\n'
  hold += f'The angent recieves the response...\n"{resp}"\n\n'

print_response = question + response + hold


print(print_response)


Initial query... I need to format numbers in basic Excel,  and use new version of Excel to discover patterns and trends in my data

Final response... You can format numbers in Excel by selecting the range of cells, clicking on the HOME tab, and choosing the desired number format. Excel 2010 introduces new data analysis and visualization tools, along with managed self-service business intelligence technologies, to help users make better decisions and analyze information in various ways.

Step 1:
The agent considers... "I should first learn how to format numbers in basic Excel and then explore the new features in Excel 2010 to analyze patterns and trends in my data."
The angent chooses the tool...  "ExcelBasics."
The angent queries the tool... "Learn how to format numbers in Excel."
The angent recieves the response...
"To format numbers in Excel, follow these steps:
1. Select the range of cells containing the numbers you want to format.
2. Click on the HOME tab.
3. In the Number group, c